In [ ]:
! pip install torch flask flask_cors onnx onnxscript torchvision onnxruntime

In [ ]:
import torch
import torch.nn as nn

import os
from collections import OrderedDict

from flask import Flask, request, jsonify, send_file
from flask_cors import CORS

# Model

In [59]:
class LiteMLP(nn.Module):
    def __init__(
            self, 
            input_nodes=3,
            hidden_layer_sizes=[10,7,5,5],
            output_nodes=10,    
    ):

        super(LiteMLP, self).__init__()

        self.input_nodes = input_nodes
        self.hidden_layer_sizes = hidden_layer_sizes
        self.output_nodes = output_nodes
        self.rebuild_model()

    def rebuild_model(
            self,
    ):
        
        layers = OrderedDict()
        all_sizes = [self.input_nodes] + self.hidden_layer_sizes + [self.output_nodes]

        for i in range(len(all_sizes) - 1):
            layers[f'linear_{i}'] = nn.Linear(all_sizes[i], all_sizes[i+1])

            if i < len(all_sizes) - 2:
                layers[f'activation_{i}'] = nn.ReLU()

        self.backbone = nn.Sequential(layers)

    def forward(
            self,
            x,
    ):
        return self.backbone(x)
    
    def add_node(
            self,
            layer_index,
            num_nodes_to_add=1,
    ):

        num_linear_layers = len(self.hidden_layer_sizes) + 1
        if not (0 <= layer_index < num_linear_layers):
            raise ValueError(f"layer_index must be between 0 and {num_linear_layers - 1}")
        
        if num_nodes_to_add < 1:
            return
            
        with torch.no_grad():
            if layer_index < len(self.hidden_layer_sizes):
                current_linear_idx = layer_index * 2
                next_linear_idx = (layer_index + 1) * 2

                self.hidden_layer_sizes[layer_index] += num_nodes_to_add
                
                old_layer_current = self.backbone[current_linear_idx]
                old_layer_next = self.backbone[next_linear_idx]
                
                new_size = old_layer_current.out_features + num_nodes_to_add
                new_layer_current = nn.Linear(old_layer_current.in_features, new_size)
                new_layer_next = nn.Linear(new_size, old_layer_next.out_features)

                new_layer_current.weight.data[:-num_nodes_to_add, :] = old_layer_current.weight.data
                new_layer_current.bias.data[:-num_nodes_to_add] = old_layer_current.bias.data
                
                new_layer_next.weight.data[:, :-num_nodes_to_add] = old_layer_next.weight.data
                new_layer_next.bias.data.copy_(old_layer_next.bias.data)

                nn.init.zeros_(new_layer_current.weight.data[-num_nodes_to_add:, :])
                nn.init.zeros_(new_layer_current.bias.data[-num_nodes_to_add:])
                nn.init.zeros_(new_layer_next.weight.data[:, -num_nodes_to_add:])
                
                self.backbone[current_linear_idx] = new_layer_current
                self.backbone[next_linear_idx] = new_layer_next
            else:
                output_layer_idx = layer_index * 2
                self.output_nodes += num_nodes_to_add
                
                old_output_layer = self.backbone[output_layer_idx]
                new_output_layer = nn.Linear(old_output_layer.in_features, self.output_nodes)
                
                new_output_layer.weight.data[:-num_nodes_to_add, :] = old_output_layer.weight.data
                new_output_layer.bias.data[:-num_nodes_to_add] = old_output_layer.bias.data
                nn.init.zeros_(new_output_layer.weight.data[-num_nodes_to_add:, :])
                nn.init.zeros_(new_output_layer.bias.data[-num_nodes_to_add:])
                
                self.backbone[output_layer_idx] = new_output_layer

    def remove_node(
            self,
            layer_index,
            num_nodes_to_remove=1,
    ):
        
        num_linear_layers = len(self.hidden_layer_sizes) + 1
        if not (0 <= layer_index < num_linear_layers):
            raise ValueError(f"layer_index must be between 0 and {num_linear_layers - 1}")
        
        if num_nodes_to_remove < 1:
            return
        
        node_indices_to_remove = list(range(num_nodes_to_remove))

        with torch.no_grad():
            if layer_index < len(self.hidden_layer_sizes):
                if self.hidden_layer_sizes[layer_index] <= num_nodes_to_remove:
                    return 

                current_linear_idx = layer_index * 2
                next_linear_idx = (layer_index + 1) * 2
                
                old_layer_current = self.backbone[current_linear_idx]
                old_layer_next = self.backbone[next_linear_idx]
                
                self.hidden_layer_sizes[layer_index] -= num_nodes_to_remove
                
                new_size = old_layer_current.out_features - num_nodes_to_remove
                new_layer_current = nn.Linear(old_layer_current.in_features, new_size)
                new_layer_next = nn.Linear(new_size, old_layer_next.out_features)
                
                mask = torch.ones(old_layer_current.out_features, dtype=torch.bool)
                mask[node_indices_to_remove] = False
                
                new_layer_current.weight.data = old_layer_current.weight.data[mask, :]
                new_layer_current.bias.data = old_layer_current.bias.data[mask]
                
                new_layer_next.weight.data = old_layer_next.weight.data[:, mask]
                new_layer_next.bias.data.copy_(old_layer_next.bias.data)

                self.backbone[current_linear_idx] = new_layer_current
                self.backbone[next_linear_idx] = new_layer_next
                
            else:
                if self.output_nodes <= num_nodes_to_remove:
                    return

                output_layer_idx = layer_index * 2
                old_output_layer = self.backbone[output_layer_idx]
                
                self.output_nodes -= num_nodes_to_remove

                new_output_layer = nn.Linear(old_output_layer.in_features, self.output_nodes)
                
                mask = torch.ones(old_output_layer.out_features, dtype=torch.bool)
                mask[node_indices_to_remove] = False
                
                new_output_layer.weight.data = old_output_layer.weight.data[mask, :]
                new_output_layer.bias.data = old_output_layer.bias.data[mask]

                self.backbone[output_layer_idx] = new_output_layer

    def add_layer(
            self, 
            layer_index, 
            new_layer_size=16
    ):

        num_hidden_layers = len(self.hidden_layer_sizes)
        if not (0 <= layer_index <= num_hidden_layers):
            raise ValueError(f"layer_index for add_layer must be between 0 and {num_hidden_layers}.")
        
        with torch.no_grad():
            
            all_modules = list(self.backbone.children())

            if layer_index == 0:
                prev_out_features = self.input_nodes
                next_layer = all_modules[0]
            else:
                prev_layer = all_modules[(layer_index - 1) * 2]
                prev_out_features = prev_layer.out_features

                if layer_index < num_hidden_layers:
                    next_layer = all_modules[layer_index * 2]
                else:
                    next_layer = all_modules[-1]
            
            new_layer = nn.Linear(prev_out_features, new_layer_size)
            new_relu = nn.ReLU()

            old_next_layer_out_features = next_layer.out_features
            modified_next_layer = nn.Linear(new_layer_size, old_next_layer_out_features)

            insert_idx = layer_index * 2
            
            all_modules.insert(insert_idx, new_layer)
            all_modules.insert(insert_idx + 1, new_relu)
            
            old_next_layer_idx = insert_idx + 2 
            all_modules[old_next_layer_idx] = modified_next_layer

            self.hidden_layer_sizes.insert(layer_index, new_layer_size)
            
            new_layers_dict = OrderedDict()
            linear_count = 0
            relu_count = 0
            for module in all_modules:
                if isinstance(module, nn.Linear):
                    new_layers_dict[f'linear_{linear_count}'] = module
                    linear_count += 1
                elif isinstance(module, nn.ReLU):
                    new_layers_dict[f'relu_{relu_count}'] = module
                    relu_count += 1
            
            self.backbone = nn.Sequential(new_layers_dict)

    def remove_layer(self, layer_index):

        num_hidden_layers = len(self.hidden_layer_sizes)
        if not (0 <= layer_index < num_hidden_layers):
            raise ValueError(f"layer_index for remove_layer must be between 0 and {num_hidden_layers - 1}.")
        if num_hidden_layers <= 1:
            return
    
        self.hidden_layer_sizes.pop(layer_index)
        self.rebuild_model()

    def change_activation(
            self,
            layer_index,
            new_activation_name,
    ):

        supported_activations = {
            'relu': nn.ReLU,
            'sigmoid': nn.Sigmoid,
            'tanh': nn.Tanh,
            'leakyrelu': nn.LeakyReLU
        }
        
        if not (0 <= layer_index < len(self.hidden_layer_sizes)):
            raise ValueError(f"layer_index must be between 0 and {len(self.hidden_layer_sizes) - 1}.")
        
        activation_key = new_activation_name.lower()
        activation_layer_name = f'activation_{layer_index}'
        
        if activation_layer_name not in self.backbone._modules:
            return
            
        NewActivationClass = supported_activations[activation_key]
        new_activation_layer = NewActivationClass()
        
        self.backbone._modules[activation_layer_name] = new_activation_layer

    def get_architecture(
            self,
    ):
        
        activations = []
        for i in range(len(self.hidden_layer_sizes)):
            activation_layer_name = f'activation_{i}'
            if activation_layer_name in self.backbone._modules:
                layer = self.backbone._modules[activation_layer_name]
                activations.append(layer.__class__.__name__)
            else:
                activations.append("None")

        return {
            "input_nodes": self.input_nodes,
            "hidden_layer_sizes": self.hidden_layer_sizes,
            "output_nodes": self.output_nodes,
            "activations": activations,
        }


In [60]:
def export_model_to_onnx(model, path):

    dummy_input = torch.randn(1, model.input_nodes)

    torch.onnx.export(
        model,
        dummy_input,
        path,
        export_params=True,
        opset_version=12,
        do_constant_folding=False,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
    )

In [61]:
def load_model(onnx_path, state_dict_path):

    if os.path.exists(state_dict_path):

        state_dict = torch.load(state_dict_path)
        input_nodes = state_dict['backbone.linear_0.weight'].shape[1]
        output_nodes = state_dict[list(state_dict.keys())[-1]].shape[0]
        hidden_layer_sizes = []

        num_hidden_layers = (len(state_dict.keys()) // 2) -1
        for i in range(num_hidden_layers):
            hidden_layer_sizes.append(state_dict[f'backbone.linear_{i}.weight'].shape[0])
            
        model = LiteMLP(input_nodes, hidden_layer_sizes, output_nodes)
        model.load_state_dict(state_dict)
    else:
        model = LiteMLP()
        
    export_model_to_onnx(model, onnx_path)
    torch.save(model.state_dict(), state_dict_path)
    return model

# Server

In [63]:
onnx_path = "lite_mlp.onnx"
state_dict_path = "lite_mlp.pth"

model = load_model(onnx_path, state_dict_path)
model.eval()

app = Flask(__name__)
CORS(app)

@app.route("/get_architecture", methods=["GET"])
def get_architecture():
    return jsonify({"status": "success", "architecture": model.get_architecture()})

@app.route("/get_model", methods=["GET"])
def get_model():
    try:
        return send_file(
                    onnx_path, 
                    mimetype="application/octet-stream"
               )
    except:
        return jsonify({"error": "File not found"}), 404
    
@app.route("/add_node", methods=["POST"])
def add_node():
    data = request.json
    layer_index = data["layer_index"]
    num_nodes = data.get("num_nodes", 1) 
    model.add_node(layer_index, num_nodes)
    torch.save(model.state_dict(), state_dict_path)
    export_model_to_onnx(model, onnx_path)
    return jsonify({"status": "success", "message": f"{num_nodes} node(s) added", "architecture": model.get_architecture()})

@app.route("/remove_node", methods=["POST"])
def remove_node():
    data = request.json
    layer_index = data["layer_index"]
    num_nodes = data.get("num_nodes", 1)    
    model.remove_node(layer_index, num_nodes)
    torch.save(model.state_dict(), state_dict_path)
    export_model_to_onnx(model, onnx_path)
    return jsonify({"status": "success", "message": f"{num_nodes} node(s) removed", "architecture": model.get_architecture()})

@app.route("/add_layer", methods=["POST"])
def add_layer():
    data = request.json
    model.add_layer(data["layer_index"], data["new_layer_size"])
    torch.save(model.state_dict(), state_dict_path)
    export_model_to_onnx(model, onnx_path)
    return jsonify({"status": "success", "message": "layer added", "architecture": model.get_architecture()})

@app.route("/remove_layer", methods=["POST"])
def remove_layer():
    data = request.json
    model.remove_layer(data["layer_index"])
    torch.save(model.state_dict(), state_dict_path)
    export_model_to_onnx(model, onnx_path)
    return jsonify({"status": "success", "message": "layer removed", "architecture": model.get_architecture()})

@app.route("/change_activation", methods=["POST"])
def change_activation():
    data = request.json
    model.change_activation(data["layer_index"], data["new_activation_name"])
    torch.save(model.state_dict(), state_dict_path)
    export_model_to_onnx(model, onnx_path)
    return jsonify({"status": "success", "message": "activation changed", "architecture": model.get_architecture()})

@app.route("/forward_pass", methods=["POST"])
def forward_pass():
    data = request.json
    input_data = data.get("input_data", [0.5] * model.input_nodes)
    
    activations = []
    hooks = []

    def hook_fn(module, input, output):
        activations.append(output.detach())

    for layer in model.backbone.children():
        hook = layer.register_forward_hook(hook_fn)
        hooks.append(hook)

    input_tensor = torch.FloatTensor([input_data])
    with torch.no_grad():
        model(input_tensor)

    for hook in hooks:
        hook.remove()

    final_activations = [input_data]
    all_activations_data = [act.tolist()[0] for act in activations]
    
    num_hidden_layers = len(model.hidden_layer_sizes)
    for i in range(num_hidden_layers):
        activation_output_idx = i * 2 + 1
        if activation_output_idx < len(all_activations_data):
            final_activations.append(all_activations_data[activation_output_idx])
    
    if all_activations_data:
        final_activations.append(all_activations_data[-1])


    return jsonify({"status": "success", "activations": final_activations})

@app.route("/get_node_details", methods=["POST"])
def get_node_details():
    data = request.json
    layer_index = int(data["layer_index"])
    node_index = int(data["node_index"])
    
    response_data = {
        "status": "success",
        "weights": [],
        "bias": None
    }
    
    if layer_index == 0:
        response_data["status"] = "info"
        response_data["message"] = "Input layer nodes do not have weights or biases."
        return jsonify(response_data)
        
    linear_layer_idx_in_backbone = (layer_index - 1) * 2
    target_layer = model.backbone[linear_layer_idx_in_backbone]
    weights = target_layer.weight.data[node_index, :].tolist()
    bias = target_layer.bias.data[node_index].item()
    
    response_data["weights"] = weights
    response_data["bias"] = bias
    
    return jsonify(response_data)
 
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


/var/folders/d3/31wmzwx14ns4qrv1k_kmq9lm0000gn/T/ipykernel_19398/1850939294.py:5: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.5:5001
Press CTRL+C to quit
127.0.0.1 - - [22/Aug/2025 23:24:55] "GET /get_architecture HTTP/1.1" 200 -
127.0.0.1 - - [22/Aug/2025 23:28:40] "GET /get_architecture HTTP/1.1" 200 -
127.0.0.1 - - [22/Aug/2025 23:28: